In [10]:
import pandas as pd
import numpy as np
import certifi
import ssl
import urllib.request

In [12]:
ssl_context = ssl.create_default_context(cafile=certifi.where())

In [13]:
url = 'https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv'
with urllib.request.urlopen(url, context=ssl_context) as response:
    df = pd.read_csv(response)
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df['clean_comment'].isna().sum()

In [ ]:
df[df['clean_comment'].isna()]

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df[(df['clean_comment'].str.strip() == '')]

In [ ]:
df[~df['clean_comment'].isna() & df['clean_comment'].str.contains('\n')]